# Libraries


In [ ]:
!pip install lxml

In [3]:
import requests
from lxml.html import document_fromstring
import re

# Connection 'cause I used Google colab.

In [4]:
# Google drive conection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Functions

In [13]:
# In case of more than one author:
def concatenate_authors(list):
    result_aux = ''
    for element in list:
        result_aux += ' AND ' + str(element[0])

    nchar = len(result_aux)
    result = result_aux[5:nchar]    
    return result


# ArticleID
def regex_id(string):
  pattern = re.compile(r'"\d{16}"') 
  result = pattern.match(string)
  if result:
    return  result.group()
  return None


# Paste all elements of a list into a single string
# https://www.w3resource.com/python-exercises/python-basic-exercise-27.php
def concatenate_list_data(list):
    result = ''
    for element in list:
        result += str(element)
    return result


# To save data with its important information
def save_data(TITLE, SUBTITLE, TEXT, ARTICLEID, AUTHOR, DATE, EDIT, PATH_SAVE):
  text_final = TITLE + '. ' + SUBTITLE + '. ' + TEXT

  # Write line to file
  name_save = str(ARTICLEID) + '_' + AUTHOR + '_' + DATE + '_' + EDIT  + '.txt' 
  
  with open(PATH_SAVE + name_save, 'w') as writefile:
    writefile.write(text_final)


def scrap_newspage_save_file(WEBPAGE, PATH_SAVE):
  resposta = requests.get(WEBPAGE)
  html = resposta.content
  # Encoding
  html = html.decode("UTF-8")
  # Tree
  arvore = document_fromstring(html)

  # Title tag
  tags_head = arvore.xpath("//title")
  title_info = []
  for tag in tags_head:
    tit = tag.xpath("text()")[-0]
    tit = str(tit)
    if re.findall('\d{2}/\d{2}/\d{4}', tit):
      title_info.append(tit) 
  
  # Get some parameters
  param = title_info[0].split("-")
  
  title = param[0].strip()
  date = param[1].strip()
  pattern = re.compile(r"/")
  date = pattern.sub("-", date)
  edit = param[2].strip()

  # Subtitle
  ## <meta name="description" content="José Aparecido Alves Filho havia sido condenado a 21 anos por latrocínio">
  tags_subtit = arvore.xpath('//meta[@name="description"]')

  subtitle = []
  for tag in tags_subtit:
    subtit = tag.attrib.get("content")
    subtitle.append(subtit)

  subtitle = subtitle[0].strip()

  # Author
  tags_author = arvore.xpath('//div[@class="c-signature"]/strong/a')

  author_aux = []
  for tag in tags_author:
    assin = tag.xpath("text()")
    author_aux.append(assin)

  # In case of more than one author:
  author = concatenate_authors(author_aux) 

  # Article ID
  html_split = html.split('"articleId": ')
  articleId_aux = html_split[1]
  articleId = regex_id(articleId_aux)[1:16] 

  # Text
  tags_p = arvore.xpath('//div[@class="c-news__body"]//p')
  paragrafo = []

  for tag in tags_p:
    parag = tag.xpath("text()")
    if len(parag):
      parag_final = concatenate_list_data(parag)
      paragrafo.append(parag_final)

  text = concatenate_list_data(paragrafo) 

  # Save
  save_data(title, 
          subtitle, 
          text, 
          articleId, 
          author, 
          date, 
          edit, 
          PATH_SAVE)

# Apply the function to data

In [14]:
scrap_newspage_save_file("https://www1.folha.uol.com.br/cotidiano/2022/01/apos-reportagem-da-folha-justica-absolve-lavrador-que-ficou-preso-por-7-anos.shtml", 
                         '/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/Processed/')

In [15]:
scrap_newspage_save_file("https://www1.folha.uol.com.br/mundo/2022/01/coreia-do-norte-diz-que-fez-novo-teste-de-missil-hipersonico.shtml?origin=folha", 
                         '/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/Processed/')